Imports

In [1]:
from pyspark.find_spark_home import _find_spark_home
from pyspark.sql import SparkSession
from pyspark import SparkConf, StorageLevel
from pyspark.sql.functions import col, count, when, lit
import os

print(_find_spark_home())

C:\Users\f\anaconda3\envs\BigDataProject\Lib\site-packages\pyspark


Setting up PySpark configurations

In [2]:
conf = (
    SparkConf()
    .setMaster("local[*]")
    .set("spark-local-dir", "C:\\spark-temp")
    .set("spark.driver.memory", "8g")
    .set("spark.executor.memory", "6g")
    .set("spark.driver.maxResultSize", "4g")
    .set(
        "spark.pyspark.python",
        "C:\\Users\f\\anaconda3\\envs\\BigDataProject\python.exe",
    )
    .set(
        "spark.pyspark.driver.python",
        "C:\\Users\f\\anaconda3\\envs\\BigDataProject\python.exe",
    )
)


spark = SparkSession.builder.appName("CovidAgeComparison").config(conf=conf).getOrCreate()

sc = spark.sparkContext

for item in sc.getConf().getAll():
    print(item)

('spark.app.name', 'CovidAgeComparison')
('spark.pyspark.driver.python', 'C:\\Users\x0c\\anaconda3\\envs\\BigDataProject\\python.exe')
('spark.driver.port', '53688')
('spark.driver.host', '192.168.0.14')
('spark.app.id', 'local-1703092586496')
('spark.executor.id', 'driver')
('spark.executor.memory', '6g')
('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-o

Loading CSV file from HDFS

In [7]:
# path to CSV file in HDFS
csv_file_path = (
    "hdfs://localhost:9000/mydataset/COVID-19_Case_Surveillance_Public_Use_Data.csv"
)

# Load the CSV file into a DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

Primary exploration of the dataset

In [8]:
df.show()

+---------------------+-------------+-----------+----------+--------------------+------+-------------+-----------------------+-------+-------+--------+----------+
|cdc_case_earliest_dt |cdc_report_dt|pos_spec_dt|  onset_dt|      current_status|   sex|    age_group|race_ethnicity_combined|hosp_yn| icu_yn|death_yn|medcond_yn|
+---------------------+-------------+-----------+----------+--------------------+------+-------------+-----------------------+-------+-------+--------+----------+
|           2022/11/10|   2022/11/11|       null|2022/11/10|Laboratory-confir...|Female|20 - 29 Years|    White, Non-Hispanic|     No|Missing|      No|   Missing|
|           2022/09/01|   2022/09/01|       null|2022/09/01|Laboratory-confir...|Female|20 - 29 Years|    White, Non-Hispanic|     No|Missing|      No|   Missing|
|           2020/12/28|   2021/01/07|       null|2020/12/28|Laboratory-confir...|Female|20 - 29 Years|    White, Non-Hispanic|     No|Missing|      No|   Missing|
|           2021/12/18

In [4]:
# Counting the amount of instances we have
df.count()

101766479

In [5]:
# Exploring the types of columns we have in the dataset
df.printSchema()

root
 |-- cdc_case_earliest_dt : string (nullable = true)
 |-- cdc_report_dt: string (nullable = true)
 |-- pos_spec_dt: string (nullable = true)
 |-- onset_dt: string (nullable = true)
 |-- current_status: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age_group: string (nullable = true)
 |-- race_ethnicity_combined: string (nullable = true)
 |-- hosp_yn: string (nullable = true)
 |-- icu_yn: string (nullable = true)
 |-- death_yn: string (nullable = true)
 |-- medcond_yn: string (nullable = true)



In [6]:
# Exploring current_status column
df.groupBy(col("current_status")).agg(count("current_status").alias("count")).show()

+--------------------+--------+
|      current_status|   count|
+--------------------+--------+
|       Probable Case|17862700|
|Laboratory-confir...|83903779|
+--------------------+--------+

